## **Step 3: Classifying high mountain wetlands**

Connect to Google Earth Engine

In [ ]:
# import Google Earth Engine API
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='...')

In [ ]:
#import geemap #for plotting interactive maps (includes folium)
from ee import batch
import pandas as pd
import geopandas as gpd
import numpy as np
import math
import folium
import os
from math import ceil

# **Import data**

In [ ]:
# Mountain areas to cut the global images - regions for upscaling
himalaya = ee.Geometry.Rectangle(62, 20, 110, 45)
alps = ee.Geometry.Rectangle(4, 43, 15, 48)
rockyMountains = ee.Geometry.Rectangle(-120, 30, -102, 48)
andes = ee.Geometry.Rectangle(-80, -40, -60, 11)

MountainRegions = ee.FeatureCollection([himalaya, alps, rockyMountains, andes])
MountainRegions_mask = himalaya.union(alps).union(rockyMountains).union(andes)

In [ ]:
# import satellite data available in the google earth engine data catalog for the four major mountain regions
ecoregions_1 = ee.FeatureCollection("RESOLVE/ECOREGIONS/2017") # RESOLVE dataset with 846 global ecoregions
dem = ee.Image("USGS/SRTMGL1_003") # Nasa DEM 30m resolution

In [ ]:
# Mountain areas to cut the global images and to use as regions for upscaling
himalaya_dem = dem.clip(himalaya)
himalaya_mask = himalaya_dem.gt(4500)
himalaya_dem_masked = himalaya_dem.updateMask(himalaya_mask)

alps_dem = dem.clip(alps)
alps_mask = alps_dem.gt(2000)
alps_dem_masked = alps_dem.updateMask(alps_mask)

rockyMountains_dem = dem.clip(rockyMountains)
rockyMountains_mask = rockyMountains_dem.gt(2000)
rockyMountains_dem_masked = rockyMountains_dem.updateMask(rockyMountains_mask)

andes_dem = dem.clip(andes)
andes_mask = andes_dem.gt(3500)
andes_dem_masked = andes_dem.updateMask(andes_mask)

# mask for all high mountain regions (DEM > 2000 masl)
dem_clipped = dem.clipToCollection(MountainRegions)
mountain_mask = dem_clipped.gt(2000)

In [ ]:
# cut the ecoregions to the mask (move this part also to the loop for training data collection)
def clip_feature(feature):
    return feature.intersection(MountainRegions_mask, ee.ErrorMargin(5))

ecoregions = ecoregions_1.map(clip_feature) #clips to exact extend of MountainRegion mask - use for upscaling.

In [ ]:
# clip all DEM derived input images to mountain regions
elevation_temp = dem_clipped.select('elevation') # elevation clipped to MountainRegions extent
slope_temp = ee.Terrain.slope(dem_clipped) # slope clipped to MountainRegions extent

# **Define functions**

Cloud masking

In [ ]:
# write a function for cloud masking
def cloudless(image):
  qa = image.select('QA60')
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  mask_clouds = qa.bitwiseAnd(cloudBitMask).eq(0).And(
      qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask_clouds).divide(10000)

###
# function that computes all vegetation indices and adds them as a band
def spectral(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
  ndwi = image.normalizedDifference(['B3', 'B8']).rename('ndwi')

  # TCG
  tcg = image.expression(
    '-0.2941*BLUE - 0.243*GREEN - 0.5424*RED + 0.7276*NIR + 0.0713*SWIRI - 0.1608*SWIRII',{
      'BLUE': image.select('B2'),
      'GREEN': image.select('B3'),
      'RED': image.select('B4'),
      'NIR': image.select('B8'),
      'SWIRI': image.select('B11'),
      'SWIRII': image.select('B12'),
    }).rename('tcg')

  # ARI
  ari = image.expression(
    '(B8 / B2) - (B8 / B3)', {
        'B8': image.select(['B8']),
        'B2': image.select(['B2']),
        'B3': image.select(['B3']),
      }).rename('ari')

  # PSRI
  psri = image.expression(
    '(B4 - B2) / B5', {
      'B4': image.select(['B4']),
      'B2': image.select(['B2']),
      'B5': image.select(['B5']),
      }).rename('psri')


  # REIP
  reip = image.expression(
    '702 + 40*((((RED + RE3)/2) - RE1) / (RE2 - RE1))', {
      'RE1': image.select(['B5']),
      'RE2': image.select(['B6']),
      'RE3': image.select(['B7']),
      'RED': image.select(['B4']),
      }).rename('reip')

  # add indices to image collection
  indices = image.addBands(ndvi).addBands(ndwi).addBands(tcg).addBands(ari).addBands(reip).addBands(psri)
  return indices

###
# function for mask
def clip_image(image):
    return image.clipToCollection(mask)
###
# mask for angle selection of sentinel-1 data. Selecting images taken with angles between 30-45 degrees.
def mask_ang_gt_30(image):
    ang = image.select(['angle'])
    return image.updateMask(ang.gt(30))
def mask_ang_lt_45(image):
    ang = image.select(['angle'])
    return image.updateMask(ang.lt(45))
###
# Function to calculate gamma0 and NDPI
def calculate_gamma0_and_ndpi(image):
    # Get the incidence angle in radians
    angle = image.select('angle')
    angle_rad = angle.multiply(math.pi / 180)

    # Calculate gamma0 for VV and VH
    gamma0_vv = image.select('VV').divide(angle_rad.cos()).rename('gamma0_VV')
    gamma0_vh = image.select('VH').divide(angle_rad.cos()).rename('gamma0_VH')

    # Calculate NDPI
    ndpi = gamma0_vv.subtract(gamma0_vh).divide(gamma0_vv.add(gamma0_vh)).rename('NDPI')

    return image.addBands([gamma0_vv, gamma0_vh, ndpi])

###
# Define the function to apply the convolution filter.Define the 3x3 boxcar kernel.
# used to create a circular kernel with a specified radius, unit, and normalization
boxcar = ee.Kernel.circle(radius=3, units='pixels', normalize=True)

def apply_filter(image):
    filtered_image = image.convolve(boxcar)
    return filtered_image

###
# Define the function to apply the angle correction and convert to gamma0.
def to_gamma0(image):
    # Select the 'angle' band and apply the angle correction
    angle_rad = image.select('angle').multiply(ee.Number(math.pi).divide(180.0))
    cos_angle = angle_rad.cos()
    correction_factor = cos_angle.log10().multiply(10.0)

    # Apply the correction to 'VV' band
    vv_corrected = image.select('VV').subtract(correction_factor)
    vh_corrected = image.select('VH').subtract(correction_factor)

    # Return the image with corrected 'VV' band
    return image.addBands(vv_corrected.rename('VV_gamma0')).addBands(vh_corrected.rename('VH_gamma0'))

###
# Function to compute the Sigma Lee filter
def sigma_lee(image, kernel_size=3, sigma=0.9):
    # Compute the local mean and variance
    reducer = ee.Reducer.mean().combine(
        reducer2=ee.Reducer.variance(),
        sharedInputs=True
    )

    # Compute the mean and variance for each pixel within the window
    stats = image.reduceNeighborhood(
        reducer=reducer,
        kernel=ee.Kernel.square(kernel_size // 2),
        optimization='window'
    )

    mean = stats.select(0)
    variance = stats.select(1)

    # Compute the noise variance
    noise_variance = variance.sqrt().divide(mean).pow(2).multiply(sigma)

    # Compute the coefficient of variation
    coef_variation = variance.sqrt().divide(mean)

    # Compute the filtered value
    one = ee.Image.constant(1)
    filter_value = one.subtract(noise_variance.divide(variance)).multiply(image.subtract(mean)).add(mean)

    # Mask invalid values
    filter_value = filter_value.updateMask(coef_variation.lte(sigma))

    return filter_value

###
# Function to extract spectral information at training points
def extract_data(feature):
    return feature.set(input.select(bands).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=feature.geometry(),
        scale=30  # Spatial resolution in meters
    ))

# **Get Sentinel images and filter by date and cloud cover**

In [ ]:
# Rasterize the 'ECO_ID' property into the image
eco_id_image = ecoregions.reduceToImage(
    properties=['ECO_ID'],
    reducer=ee.Reducer.first()
)

# **Prepare training and validation data sets from full training data set**

Separate training and testing datasets

In [ ]:
bands = ['ndvi', 'ndwi', 'elevation', 'tpi', 'slope', 'tcg', 'NDPI', 'VV_gamma0', 'VH_gamma0', 'reip', 'ari', 'ecoregion']
target = 'class'

In [ ]:
# Filter the training data to remove features with null values in any of the properties
# load CSV table created in step-1 from GEE assets
training_data_1 = ee.FeatureCollection('projects/.../assets/training_data_1')
training_data_2 = ee.FeatureCollection('projects/.../assets/training_data_2')
# join the training data
training_data = training_data_1.merge(training_data_2)

training_data = training_data.filter(ee.Filter.notNull(bands))
training_data = training_data.randomColumn('random')

trainSet = training_data.filter(ee.Filter.lte('random', 0.8)) # 2/3 of data for training //80%
testSet = training_data.filter(ee.Filter.gt('random', 0.8)) # 1/3 of data for validation //20%

# **Train Random Forest Classifier**

In [ ]:
# train the classifier
# Number of features
numFeatures = len(bands)

# Create and train the smileRandomForest classifier
classifier_prob = ee.Classifier.smileRandomForest(
    numberOfTrees=50,
    variablesPerSplit=int(numFeatures ** 0.5)
).setOutputMode('PROBABILITY')

classifier_prob = classifier_prob.train(trainSet, 'class', bands)

# **Apply classifier to a different year and region of interest**

# **Apply when processing the Rocky Mountains, High Mountain Asia and the Alps**

In [1]:
start_date = ['2019-06-01','2020-06-01','2021-06-01','2022-06-01','2023-06-01','2024-06-01'] # same start days for all 6 years (2019-2024)
end_date = ['2019-10-31','2020-10-31','2021-10-31','2022-10-31','2023-10-31','2024-10-31'] # same end days for all 6 years (2019-2024)

In [ ]:
# set upscale mask
mask_upscale = alps
#mask_upscale = rockyMountains
#mask_upscale = himalaya

In [ ]:
### !!! use when processing the Rocky Mountains, High Mountain Asia and the Alps !!! ###
# sentinel 2
#collection = ee.FeatureCollection([])
for start, end in zip(start_date, end_date):
  s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
               .filterBounds(mask_upscale)\
               .filterDate(start, end)\
               .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 30))\
               .map(cloudless)
  s2_clipped = s2.map(lambda image: image.clip(mask_upscale))
  #collection = collection.merge(s2_clipped)
s2_m = s2_clipped.median()

# get sentinel 1 image
s1_pol = ee.ImageCollection('COPERNICUS/S1_GRD') \
           .filterBounds(mask_upscale)\
           .filterDate(start, end)\
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
           .filter(ee.Filter.eq('resolution_meters', 10))
s1_pol = s1_pol.map(lambda image: image.clip(mask_upscale))

# **Apply when processing the Andes**

In [ ]:
start_date = '2019-01-01'
end_date = '2024-12-31'

In [ ]:
# set upscale mask
mask_upscale = andes

In [ ]:
### !!! use when processing the Andes !!! ###
# sentinel 2
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
            .filterBounds(mask_upscale)\
            .filterDate(start_date, end_date)\
            .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10))\
            .map(cloudless)
s2_clipped = s2.map(lambda image: image.clip(mask_upscale))
s2_m = s2_clipped.median()

# get sentinel 1 image
s1_pol = ee.ImageCollection('COPERNICUS/S1_GRD') \
           .filterBounds(mask_upscale)\
           .filterDate(start_date, end_date)\
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
           .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
           .filter(ee.Filter.eq('resolution_meters', 10))
s1_pol = s1_pol.map(lambda image: image.clip(mask_upscale))

In [ ]:
# Apply functions to s1
s1_pol1 = s1_pol.map(mask_ang_gt_30)
s1_pol1 = s1_pol1.map(mask_ang_lt_45)

NDPI = s1_pol1.map(calculate_gamma0_and_ndpi)
NDPI = NDPI.map(apply_filter)
NDPI = NDPI.median()

s1_pol2 = s1_pol.map(to_gamma0)
s1_pol_lee_filtr = s1_pol2.map(sigma_lee)

VV_VH = s1_pol_lee_filtr.mean()
VVsd_VHsd = s1_pol_lee_filtr.reduce(ee.Reducer.stdDev())

# topographic position index (TPI)
focal_mean = elevation_temp.clip(mask_upscale).focalMean(5)
tpi = elevation_temp.clip(mask_upscale).subtract(focal_mean)
tpi = tpi.rename(['tpi'])

# clip flow accumulation, slope and elevation
elevation = elevation_temp.clip(mask_upscale)
slope = slope_temp.clip(mask_upscale)

# sentinel 2 indices
spec_indices = spectral(s2_m)

# ecoregions
ecoregion = eco_id_image.clip(mask_upscale)
ecoregion = ecoregion.rename(['ecoregion'])

indices = spec_indices.addBands(slope).addBands(elevation).addBands(tpi).addBands(VV_VH).addBands(VVsd_VHsd).addBands(NDPI).addBands(ecoregion)#.addBands(pc1)

bands = ['ndvi', 'ndwi', 'elevation', 'tpi', 'slope', 'tcg', 'NDPI', 'VV_gamma0', 'VH_gamma0', 'reip', 'ari', 'ecoregion']#, 'pc1']
input_yoi = indices.select(bands)

# **Classify period of interest**

# **Get probabilities and export the images**

In [ ]:
# apply probability classifier
classified_prob = input_yoi.classify(classifier_prob)

# Define the class of interest and probability threshold
class_name = 'classification'
probability = classified_prob.select(class_name)# Extract probability for the specific class
high_prob = probability.gt(0.70)# Threshold the probability to keep only high-probability areas

# Mask the original image using the high probability threshold
high_prob_image = classified_prob.updateMask(high_prob)

In [ ]:
final_prob_image = high_prob_image.unitScale(0,1).multiply(255).toByte()

In [ ]:
# !!! change mask when running for a different mountain range
final_prob_image_clipped = final_prob_image.updateMask(alps_mask)

# **Export results to drive**

In [ ]:
#export to drive
region = alps
image = final_prob_image_clipped

def create_grid(geometry, max_dimension):
    """Divide the geometry into smaller tiles."""
    # Get the coordinates of the region.
    if isinstance(geometry, ee.FeatureCollection):
        geometry = geometry.geometry()

    bounds = geometry.bounds().coordinates().get(0).getInfo()
    lon_min, lat_min = bounds[0][0], bounds[0][1]
    lon_max, lat_max = bounds[2][0], bounds[2][1]

    # Calculate the number of tiles needed, always round up.
    lon_diff = lon_max - lon_min
    lat_diff = lat_max - lat_min

    num_lon_tiles = int(ceil(lon_diff / max_dimension))
    num_lat_tiles = int(ceil(lat_diff / max_dimension))

    # Create the tiles.
    tiles = []
    for i in range(num_lon_tiles):
        for j in range(num_lat_tiles):
            tile = ee.Geometry.Rectangle([
                lon_min + i * max_dimension,
                lat_min + j * max_dimension,
                min(lon_min + (i + 1) * max_dimension, lon_max),
                min(lat_min + (j + 1) * max_dimension, lat_max)
            ])
            tiles.append(tile)
    return tiles

# Create tiles with a maximum dimension of X degrees.
tiles = create_grid(region, max_dimension=2)

In [ ]:
# Export each tile
if isinstance(region, ee.FeatureCollection):
    region = region.geometry()

for i, tile in enumerate(tiles):
    # Ensure the intersection geometry is valid
    intersection = region.intersection(tile, ee.ErrorMargin(1))

    # Clip the image to the intersection geometry
    clipped_tile = image.clip(intersection)

    # Get the bounds of the tile for export
    tile_bounds = tile.bounds().getInfo()['coordinates']

    # Example: export the clipped tile to Google Drive
    task = ee.batch.Export.image.toDrive(
        image=clipped_tile,
        description=f'Wetlands_2019_2024_alps_70_prob_{i}',
        scale=30,
        region=tile_bounds,
        fileFormat = 'GeoTIFF',
        folder='earth_engine_exports_alps'
    )
    task.start()

    print(f'Started export task for tile {i}')